# Time Series Forecasting for Corporation Favorita 

### **Project Overview**

The primary objective of this project is to build a predictive model that accurayely predicts the unit sales for thousands of items sold at different Favorita stores

The CRISP-DM Framework would be used for this project.

The CRISP-DM (Cross-Industry Standard Process for Data Mining) framework is a widely used methodology for data mining and data analysis projects. It provides a structured approach for planning and executing data projects. The process is cyclic and iterative, consisting of six phases:

1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

## Business Understanding
### Hypothesis

In [2]:

import numpy as np
import pyodbc
from dotenv import dotenv_values
import statsmodels.api as sm
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

#create connection string
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER={dap-projects-database.database.windows.net};DATABASE={dapDB};UID={learning_project_3};PWD={A$uB1Lp3$2@24}"
connection= pyodbc.connect(connection_string)


## Data Understanding
The data for this projects has been divided into 3. 

From an SQL database management system, three tables are contained.
Within the OneDrive, two files which are just for testing and not training.
Finally, contained in a GitHub repository are additional files.





In [5]:
# query the database for the first part of the data and assign it a variable name 'df_oil'
query = "Select * from dbo.oil"

df_oil = pd.read_sql(query, connection)

C:\Users\user\AppData\Local\Temp\ipykernel_18324\2195552024.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_oil = pd.read_sql(query, connection)


In [6]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [7]:
# query the database for the second part of the data and assign it a variable name 'df_events'
query = "Select * from dbo.holidays_events"

df_events = pd.read_sql(query, connection)

C:\Users\user\AppData\Local\Temp\ipykernel_18324\3282018163.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_events = pd.read_sql(query, connection)


In [9]:
df_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [10]:
# query the database for the third part of the data and assign it a variable name 'df_stores'
query = "Select * from dbo.stores"

df_stores = pd.read_sql(query, connection)

C:\Users\user\AppData\Local\Temp\ipykernel_18324\392509437.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stores = pd.read_sql(query, connection)


In [11]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [12]:
# reading data from GitHub repository
df_train= pd.read_csv(r"C:\Users\user\Documents\New folder\LP.3\store-sales-forecasting\train.csv")
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [13]:
# Reading the second data from GitHub repository
df_trans= pd.read_csv(r"C:\Users\user\Documents\New folder\LP.3\store-sales-forecasting\transactions.csv")
df_trans

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [14]:
# Create a dictionary of the datasets
data = {'holiday': df_events, 'stores': df_stores, 'oil': df_oil, 'transactions': df_trans, 'train': df_train}

In [15]:
# Check the datatypes and the presence of missing values in each of the datasets
# Using '\033[1mtext\033[0m' to make text bold
for df_name, dataset in data.items():
    print(f'\033[1mFor {df_name} dataset\033[0m:')
    dataset.info()
    print('_'*45)
    print('\n')

For holiday dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
_____________________________________________


For stores dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
____

In [16]:
# Check the shape, and the presence of missing values and duplicates in each of the datasets
# Use '\033[1mtext\033[0m' to make text bold
for df_name, dataset in data.items():
    print(f'\033[1mFor {df_name} dataset\033[0m')
    print(f'Shape: {dataset.shape}')
    print(f'Missing values = {dataset.isna().sum().sum()}')
    print(f'Duplicates = {dataset.duplicated().sum()}')
    print('_'*30)
    print('\n')

For holiday dataset
Shape: (350, 6)
Missing values = 0
Duplicates = 0
______________________________


For stores dataset
Shape: (54, 5)
Missing values = 0
Duplicates = 0
______________________________


For oil dataset
Shape: (1218, 2)
Missing values = 43
Duplicates = 0
______________________________


For transactions dataset
Shape: (83488, 3)
Missing values = 0
Duplicates = 0
______________________________


For train dataset
Shape: (3000888, 6)
Missing values = 0
Duplicates = 0
______________________________




### Key Observations
- The only column with missing values is "dcoilwtico" from the oil dataset, which has 43 missing values 
 - All the date columns in the dataset are all in obect format
 - There are no duplicates in the dataset.
 - There is no column to identify the days in which there were holidays and days in which there were no holidays.

### Merging the datasets

In [33]:
# Merge df_train and df_trans on 'date' and 'store_nbr'
df1 = pd.merge(df_train, df_trans, on=['date', 'store_nbr'], how='left')

# Merge df1 with df_events on 'date'
df2 = pd.merge(df1, df_events, on='date', how='left')

# Merge df2 with df_oil on 'date'
df3 = pd.merge(df2, df_oil, on='date', how='left')

# Merge df3 with df_stores on 'store_nbr'
df4 = pd.merge(df3, df_stores, on='store_nbr', how='left')

# Display the final merged DataFrame
df4.head()


,id,date,store_nbr,family,sales,onpromotion,transactions,type_x,locale,locale_name,description,transferred,dcoilwtico,city,state,type_y,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
